<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
    <h3 style="text-align: center; background-color:rgb(18, 135, 198); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>Table of Contents </b></h3>
    
- I. Crawling Data and Merging Datasets
- II. Exploring and Pre-processing Data
    - [2.1. Basic infomation](#2.1.)
        - [2.1.1. Shape of data](#2.1.1.)
        - [2.1.2. Meaning of each row](#2.1.2.)
        - [2.1.3. Meaning of each column](#2.1.3.)
        - [2.1.4. Check duplicate](#2.1.4.)
        - [2.1.5. Data type of each column (standardize if necessary)](#2.1.5.)
        - [2.1.6. Missing values of each column](#2.1.6)
    - [2.2 Data distribution](#2)
        - [2.2.1. Numerical columns](#2.2.1.)
        - [2.2.2. Categorical columns](#2.2.2.)
    - [2.3. Abnormal values and outliers](#2.3.)
    - [2.4. Correlation between variables](#2.4)
- III. Asking meaningful questions

In [2]:
import os
import requests
from flask import Flask, session
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.cache_handler import CacheHandler
import base64

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager

In [3]:
import time
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import urllib

In [4]:
import pandas as pd
import numpy as np

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
    <h3 style="text-align: center; background-color:rgb(2, 100, 153); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1. Crawling Data and Merging Datasets </b></h3>

<a class="anchor" id="1.1."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.1. Helper functions </b></h4>

In [5]:
def get_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    
    # Correct Base64 encoding
    credentials = f"{client_id}:{client_secret}".encode('utf-8')
    encoded_credentials = base64.b64encode(credentials).decode('utf-8')
    
    auth_header = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    auth_data = {
        'grant_type': 'client_credentials'
    }
    
    response = requests.post(auth_url, headers=auth_header, data=auth_data)
    
    if response.status_code == 200:
        data = response.json()
        return data['access_token']
    else:
        print("Error:", response.status_code, response.text)
        return None


In [6]:
def refresh_access_token(client_id, client_secret, refresh_token):
    auth_url = 'https://accounts.spotify.com/api/token'
    
    # Correct Base64 encoding
    credentials = f"{client_id}:{client_secret}".encode('utf-8')
    encoded_credentials = base64.b64encode(credentials).decode('utf-8')
    
    auth_header = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    auth_data = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    response = requests.post(auth_url, headers=auth_header, data=auth_data)
    
    if response.status_code == 200:
        data = response.json()
        return data['access_token']
    
    else:
        print("Error:", response.status_code, response.text)
        return None
    

<a class="anchor" id="1.2."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.2. Define necessary params for authentication </b></h4>


In [7]:
client_id = "dbdd996b41794e00a9147d472d3b8ab2"
client_secret = "d5cc548e8c2f4d50b7e5b61fa75846af"
redirect_uri = "http://localhost:3000/callback"
# scope = "user-library-read"

In [8]:
auth_url = "https://accounts.spotify.com/authorize"
auth_params = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    # "scope": scope
}

authorization_url = f"{auth_url}?{urllib.parse.urlencode(auth_params)}"
print("Go to the following URL to authorize the application:")
print(authorization_url)

Go to the following URL to authorize the application:
https://accounts.spotify.com/authorize?client_id=dbdd996b41794e00a9147d472d3b8ab2&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2Fcallback


In [9]:
# Step 2: After the user authorizes the application, they will be redirected to the redirect_uri with a code parameter
# You need to capture that code parameter and use it to obtain an access token and refresh token

# For demonstration purposes, let's assume you have obtained the authorization code
authorization_code = input("Enter the authorization code: ")

# Step 3: Exchange the authorization code for an access token and refresh token
token_url = "https://accounts.spotify.com/api/token"
credentials = f"{client_id}:{client_secret}".encode('utf-8')
encoded_credentials = base64.b64encode(credentials).decode('utf-8')

token_headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": authorization_code,
    "redirect_uri": redirect_uri
}

response = requests.post(token_url, headers=token_headers, data=token_data)

if response.status_code == 200:
    data = response.json()
    access_token = data["access_token"]
    refresh_token = data["refresh_token"]
    print("Access Token:", access_token)
    print("Refresh Token:", refresh_token)
else:
    print("Error:", response.status_code, response.text)

# Step 4: Use the refresh token to obtain a new access token
def refresh_access_token(client_id, client_secret, refresh_token):
    auth_url = 'https://accounts.spotify.com/api/token'
    
    credentials = f"{client_id}:{client_secret}".encode('utf-8')
    encoded_credentials = base64.b64encode(credentials).decode('utf-8')
    
    auth_header = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    auth_data = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    response = requests.post(auth_url, headers=auth_header, data=auth_data)
    
    if response.status_code == 200:
        data = response.json()
        return data['access_token']
    else:
        print("Error:", response.status_code, response.text)
        return None

# Use the refresh token to get a new access token
new_access_token = refresh_access_token(client_id, client_secret, refresh_token)
print("New Access Token:", new_access_token)

Access Token: BQBKde10MRX_2PNyYvb4sV7ZSNV49FNg6VYgEwdvzsR2mF3Ei4D7QREm0rJTod-BTZex_bmQtwSMhY9W6cb52rxo-CU5-Ej1bO_gvyzS4XYS228JTMPrfAH4Y8nyUlxUFqPLtBYCNvzHlD7s35qF-vHSXTdxDqmezyDpl5oadRXrz-ufgSn-B7Wf6o6DbiyK2ZsuqZAV_AWJ_8MQyUoDXOr1r4Vo4S5Cq4O5-A
Refresh Token: AQDjM9wF0hYXhSqxs-OL6HnRO3KwreQadh9pNE_Jix_K9xHvKlCkNw5GfOhKWC3JZwj-b_IN4-vDqRE1lfOmzQBZxqS5xwIig-h_37Jkd1tT-C_KTzwdAZO09wdXmCXjqnA
New Access Token: BQBCHC6Ggsmgj8nLh8XgPlX8_OC-uI0hySb0IAyG6MJLK56bsa5qAIHk1tsTVQW6OrblJK0ow32kRVuSNV5jwJVBHAWWxxTHB5t0KhU8za96fw9U2pTjWzChJurl_qtW-G2-6ubtVdBkinomkqb-pN107nqNYNlN1D6pOShyjqr4gIZtyxrYP6zh5JwyuXSX2A2AW8w3tppo3hgnMrYW44mPMTaORioXdqpQMg


In [10]:
new_access_token
sp = spotipy.Spotify(auth=new_access_token)

<a class="anchor" id="1.3."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.3. Crawl songs by market only </b></h4>


In [ ]:
def get_vietnamese_songs(query, limit_per_request=50, total_songs=3000, market="VN", min_popularity=50):
    all_tracks = []
    track_ids = set()
    offset = 0

    while len(all_tracks) < total_songs:
        try:
            results = sp.search(q=query, limit=limit_per_request, offset=offset, type='track', market=market)

            tracks = results.get('tracks', {}).get('items', [])
            if not tracks:  # Stop if no more tracks are found
                break

            for track in tracks:
                if track['id'] in track_ids:
                    continue  # Skip if the track is already in the dataset

                if track['popularity'] < min_popularity:
                    continue  # Skip tracks that are not popular enough

                album_id = track['album']['id'] if 'album' in track else None
                album_details = sp.album(album_id) if album_id else {}
                genres = album_details.get('genres', [])
                available_markets = album_details.get('available_markets', [])
                album_popularity = album_details.get('popularity', None)
                artist_genres = [genre for artist in track['artists'] for genre in artist.get('genres', [])]
                all_tracks.append({
                    'album': track['album']['name'], 
                    'album_type': track['album']['album_type'],
                    'album_popularity': album_popularity,
                    'artists': [artist['name'] for artist in track['artists']], 
                    'available_markets': available_markets, 
                    'disc_number': track['disc_number'], 
                    'duration_ms': track['duration_ms'],
                    'explicit': track['explicit'], 
                    'id': track['id'], 
                    'is_local': track['is_local'],
                    'is_playable': track.get('is_playable', False), 
                    'name': track['name'], 
                    'popularity': track['popularity'], 
                    'track_number': track['track_number'],
                    'type': track['type'], 
                    'genres': genres,  # Now using album genres
                    'release_date': track['album']['release_date'],
                    'artist_genres': artist_genres
                })

                track_ids.add(track['id'])  # Add track ID to the set

            offset += limit_per_request
            print(f"Fetched {len(all_tracks)} songs so far...")

            # Avoid hitting rate limits
            time.sleep(2)

        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 429:
                retry_after = int(e.headers.get('Retry-After', 1))
                print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"Error: {e}")
                break  # Stop execution on critical errors

    return all_tracks

years = range(2020, 2025)
vietnamese_songs = []

for year in years:
    query = f"year:{year}"  # Corrected query format
    vietnamese_songs.extend(get_vietnamese_songs(query=query, total_songs=1000, min_popularity=1))

print(f"Total Vietnamese songs collected: {len(vietnamese_songs)}")

In [19]:
df = pd.DataFrame(vietnamese_songs)

In [20]:
df

,album,album_type,album_popularity,artists,available_markets,disc_number,duration_ms,explicit,id,is_local,is_playable,name,popularity,track_number,type,genres,release_date
0,Đi Về Nhà,single,50,"[Đen, JustaTee]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,200000,False,7LVrrX8pkzI9fMF88nGabt,False,True,Đi Về Nhà,61,1,track,[],2020-12-20
1,CUA,single,40,"[MANBO, HIEUTHUHAI]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,193475,True,0ReId5SRMspb6ISw0rBsBN,False,True,CUA,50,1,track,[],2020-04-05
2,Hẹn Gặp Em Dưới Ánh Trăng,single,53,"[MANBO, HIEUTHUHAI, HURRYKNG]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,222000,False,0s7RyyUlQfd8mnnboHe18n,False,True,Hẹn Gặp Em Dưới Ánh Trăng,63,1,track,[],2020-09-25
3,Still With You,single,64,[Jung Kook],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,239905,False,0eFMbKCRw8KByXyWBw8WO7,False,True,Still With You,75,1,track,[],2020-06-05
4,HELP EVER HURT NEVER,album,65,[Fujii Kaze],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,185573,False,0o9zmvc5f3EFApU52PPIyW,False,True,Shinunoga E-Wa,74,8,track,[],2020-05-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,külm,single,31,[An-Marlen],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,176200,False,1M00EDfnI8zrf0clHUSJY6,False,True,külm,42,1,track,[],2024-12-06
4856,SUEÑO MAGNIFICO,single,48,[Najm],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,136968,True,6x4eiL3u5iWWDVumT8JSqd,False,True,LOTSOFLOVE,51,4,track,[],2024-09-05
4857,Alleen Met Jou,single,56,"[Yves Berendse, Emma Heesters]","[AR, AU, AT, BO, BR, BG, CA, CL, CO, CR, CY, C...",1,193694,False,2K7HqkHEUfz7c4BzEO33Qd,False,True,Alleen Met Jou,66,1,track,[],2024-09-27
4858,TUTTA VITA,album,72,"[Olly, JVLI]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,190122,False,1ej2NcW0CHnscPp1RZpr1V,False,True,Devastante,68,6,track,[],2024-10-25


In [21]:
# Check for duplicates
df.duplicated(subset=['id']).sum()

np.int64(0)

In [22]:
# Check number of unique values in available_markets
df['available_markets'] = df['available_markets'].apply(len)
df['available_markets'].value_counts()

available_markets
185    3280
184     813
183     610
182      44
181      17
180      15
177      14
179      13
55        6
176       5
95        5
178       4
32        3
136       2
138       2
0         2
172       2
175       2
88        2
13        2
146       2
135       2
142       1
130       1
87        1
94        1
96        1
170       1
157       1
171       1
174       1
33        1
165       1
12        1
134       1
Name: count, dtype: int64

In [ ]:
df['genres'] = df['genres'].apply(lambda x: ', '.join(x) if x else None)
df['genres'].value_counts()

Series([], Name: count, dtype: int64)

In [32]:
df['album_popularity'].unique()

array([50, 40, 53, 64, 65, 66, 54, 41, 83, 74, 71, 47, 42, 67, 45, 75, 44,
       46, 43, 39, 49, 68, 79, 82, 72, 69, 58, 60, 63, 48, 70, 76, 77, 56,
       57, 73, 62, 61, 55, 59, 78, 38, 52, 51, 32, 33, 36, 37, 35, 80, 31,
       28, 34, 30, 26, 27, 29, 25, 20, 85, 81, 86, 24, 87, 23, 84, 88, 92,
       89])

In [24]:
df.to_csv('data/vietnamese_songs.csv', index=False)

<a class="anchor" id="1.4."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.4. Crawl songs by Vietnamese artists </b></h4>



In [ ]:
def get_song_by_artist(artist_name, limit_per_request=50, total_songs=50):
    all_tracks = []
    track_ids = set()
    offset = 0
    batch_size = 20 
    album_cache = {} 
    artist_cache = {} 

    print("Fetching song for artist: ", artist_name)
    while len(all_tracks) < total_songs:
        try:
            results = sp.search(q=f"artist:{artist_name}", limit=limit_per_request, offset=offset, type='track', market = 'VN')
            tracks = results.get('tracks', {}).get('items', [])

            if not tracks:  
                break 

            album_ids = {track['album']['id'] for track in tracks if 'album' in track}  
            new_album_ids = album_ids - album_cache.keys()
            artist_ids = {artist['id'] for track in tracks for artist in track.get('artists', []) if 'id' in artist}
            new_artist_ids = artist_ids - artist_cache.keys()

            for i in range(0, len(new_album_ids), batch_size):
                album_batch = list(new_album_ids)[i:i + batch_size]
                album_results = sp.albums(album_batch)  
                for album in album_results.get('albums', []):
                    album_cache[album['id']] = {
                        'genres': album.get('genres', []),
                        'available_markets': album.get('available_markets', []),
                        'popularity': album.get('popularity', None),
                    }

            for i in range(0, len(new_artist_ids), 50):
                artist_batch = list(new_artist_ids)[i:i + 50]
                artist_results = sp.artists(artist_batch)
                for artist in artist_results.get('artists', []):
                    artist_cache[artist['id']] = artist.get('genres', [])  

            for track in tracks:
                if track['id'] in track_ids:
                    continue  

                album_info = album_cache.get(track['album']['id'], {})
                artist_names = [artist.get('name', 'Unknown Artist') for artist in track.get('artists', [])]
                artist_genres = set()  

                for artist in track.get('artists', []):
                    artist_genres.update(artist_cache.get(artist['id'], []))

                all_tracks.append({
                    'id': track['id'], 
                    'name': track.get('name', 'Unknown Track'), 
                    'artists': artist_names,
                    'album': track['album'].get('name', 'Unknown Album'), 
                    'album_type': track['album'].get('album_type', 'unknown'),
                    'album_popularity': album_info.get('popularity'),
                    'genres': list(artist_genres), 
                    'available_markets': album_info.get('available_markets', []), 
                    'track_number': track.get('track_number', 0),
                    'disc_number': track.get('disc_number', 0), 
                    'duration_ms': track.get('duration_ms', 0),
                    'popularity': track.get('popularity', 0), 
                    'release_date': track['album'].get('release_date', 'Unknown'),
                })

                track_ids.add(track['id'])  

            offset += limit_per_request
            print(f"Fetched {len(all_tracks)} songs so far...")

            if len(all_tracks) >= total_songs:
                break  

            time.sleep(1)  

        except spotipy.exceptions.SpotifyException as e:
            if e.http_status == 429:
                retry_after = int(e.headers.get('Retry-After', 1))
                print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                time.sleep(retry_after)
            else:
                print(f"Error: {e}")
                break  

    return all_tracks


def get_vietnamese_songs(artists_list, total_songs=100, song_per_artist = 30):
    all_vietnamese_songs = []
    track_ids = set()
    for artist in artists_list:
        songs = get_song_by_artist(artist, total_songs=song_per_artist)  
        for song in songs:
            if song['id'] not in track_ids:
                all_vietnamese_songs.append(song)
                track_ids.add(song['id'])
        if len(all_vietnamese_songs) >= total_songs:
            break  
        time.sleep(1)  
    return all_vietnamese_songs


In [ ]:
genres = ['vinahouse', 'v-pop', 'vietnamese lo-fi', 'vietnam indie', 'vietnamese hip hop', 'vietnamese', 'vietnam', 'vietnamese bolero']

#Get vietnamese artist from these genre
def get_artists_for_genres(genres, limit_per_request=50, total_artists_per_genre = 100):
    
    all_artists = []
    artist_ids = set()

    for genre in genres:
        print(f"\nFetching artists for genre: {genre}")
        offset = 0
        genre_artists = []

        while len(genre_artists) < total_artists_per_genre:
            try:
                results = sp.search(q=f"genre:{genre}", type="artist", limit=limit_per_request, offset=offset)
                artists = results.get("artists", {}).get("items", [])

                if not artists:
                    print(f"No more artists found for {genre}.")
                    break  

                new_artists = [
                    artist for artist in artists 
                    if artist["id"] not in artist_ids
                ]

                for artist in new_artists[:total_artists_per_genre - len(genre_artists)]:
                    artist_info = {
                        "id": artist["id"],
                        "name": artist["name"],
                        "popularity": artist.get("popularity", 0),
                        "followers": artist["followers"].get("total", 0),
                        "genres": artist.get("genres", []),
                    }
                    genre_artists.append(artist_info)
                    artist_ids.add(artist["id"])

                offset += limit_per_request
                print(f"Fetched {len(genre_artists)} artists for {genre} so far...")

                time.sleep(1)

            except spotipy.exceptions.SpotifyException as e:
                if e.http_status == 429:
                    retry_after = int(e.headers.get("Retry-After", 1))
                    print(f"Rate limit exceeded. Retrying in {retry_after} seconds...")
                    time.sleep(retry_after)
                else:
                    print(f"Spotify API error: {e}")
                    break
            except Exception as e:
                print(f"Unexpected error: {e}")
                break

        all_artists.extend(genre_artists)

    return all_artists

In [ ]:
artists_df = get_artists_for_genres(genres=genres, limit_per_request=50, total_artists_per_genre=100)
artists_df = pd.DataFrame(artists_df)
artists_df.to_csv("data/artists.csv", index=False)


,id,name,popularity,followers,genres
0,08xn7sYY4KworP8Vc3GoiU,Orinn,55,558309,"['vinahouse', 'vietnamese lo-fi', 'v-pop', 'lo..."
1,0jF7Zlz8P5p74zcH7YwcMU,Anh Quân Idol,42,36965,"['vinahouse', 'vietnamese lo-fi', 'v-pop', 'lo..."
2,7MsHVZKh1TeKgGqm65kZnY,Vũ Duy Khánh,35,22264,"['vinahouse', 'v-pop', 'vietnamese bolero']"
3,7JpgJSzLFVxFcMIRUhq7vV,Lương Bích Hữu,36,27958,"['vinahouse', 'v-pop', 'vietnamese bolero', 'v..."
4,0okOBQvuWiPJnfvvmosphN,VIET.,35,24396,"['vinahouse', 'vietnamese lo-fi', 'v-pop']"
...,...,...,...,...,...
331,7vCKeEqgPv1FFEM2OivtNj,Ngọc Ngữ,27,2349,"['vietnamese bolero', 'bolero']"
332,5jAkbuyAcJlFqLH4z88ukv,Trần Thái Hoà,28,23977,"['vietnamese bolero', 'bolero']"
333,41eUMVb8BInRlwH1Cx6v3N,Lương Tùng Quang,28,9697,"['vietnamese bolero', 'v-pop']"
334,4oFMIlXAVelFtZxOEnc9EX,Châu Ngọc Hà,27,1795,"['vietnamese bolero', 'bolero']"


In [ ]:
artists_df = pd.read_csv("data/artists.csv")
artists_df = artists_df.sort_values('followers', ascending=False).reset_index(drop=True)
artists_df

,id,name,popularity,followers,genres
0,5dfZ5uSmzR7VQK0udbAVpf,Sơn Tùng M-TP,64,6425018,"['v-pop', 'vinahouse', 'vietnamese hip hop']"
1,1LEtM3AleYg1xabW6CRkpi,Đen,60,4255917,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
2,57g2v7gJZepcwsuwssIfZs,Vũ.,62,2357407,"['v-pop', 'vietnam indie', 'vietnamese lo-fi']"
3,5HZtdKfC4xU0wvhEyYDWiY,HIEUTHUHAI,67,2109280,"['v-pop', 'vietnamese hip hop', 'vietnamese lo..."
4,1CWwyDPjCowRTO4p6A7r6g,Mỹ Tâm,49,1974560,"['v-pop', 'vietnamese bolero', 'vietnam indie']"
...,...,...,...,...,...
331,6h54CyJrKz7VFTbhcNa5z5,Cao Sy Hung,2,155,['vinahouse']
332,63aNYEnei9zRJabFnVRQ3c,Mai Quốc Việt,0,135,['vinahouse']
333,7kuPvJW5dFp3jcDzbCJuCf,Nguyen Hai Yen,0,120,[]
334,22Cg030HR9Ytnyc1uJ11rO,Hàn Khởi,4,115,"['vinahouse', 'v-pop']"


In [ ]:
def reformat_name(name):
    return unidecode(name)

artists_df['reformated name'] = artists_df['name'].apply(reformat_name)

artists_df

,id,name,popularity,followers,genres,reformated name
0,5dfZ5uSmzR7VQK0udbAVpf,Sơn Tùng M-TP,64,6425018,"['v-pop', 'vinahouse', 'vietnamese hip hop']",Son Tung M-TP
1,1LEtM3AleYg1xabW6CRkpi,Đen,60,4255917,"['v-pop', 'vietnamese hip hop', 'vietnam indie...",Den
2,57g2v7gJZepcwsuwssIfZs,Vũ.,62,2357407,"['v-pop', 'vietnam indie', 'vietnamese lo-fi']",Vu.
3,5HZtdKfC4xU0wvhEyYDWiY,HIEUTHUHAI,67,2109280,"['v-pop', 'vietnamese hip hop', 'vietnamese lo...",HIEUTHUHAI
4,1CWwyDPjCowRTO4p6A7r6g,Mỹ Tâm,49,1974560,"['v-pop', 'vietnamese bolero', 'vietnam indie']",My Tam
...,...,...,...,...,...,...
331,6h54CyJrKz7VFTbhcNa5z5,Cao Sy Hung,2,155,['vinahouse'],Cao Sy Hung
332,63aNYEnei9zRJabFnVRQ3c,Mai Quốc Việt,0,135,['vinahouse'],Mai Quoc Viet
333,7kuPvJW5dFp3jcDzbCJuCf,Nguyen Hai Yen,0,120,[],Nguyen Hai Yen
334,22Cg030HR9Ytnyc1uJ11rO,Hàn Khởi,4,115,"['vinahouse', 'v-pop']",Han Khoi


In [ ]:
song_df = get_vietnamese_songs(artists_list=artists_df['reformated name'], total_songs=5000, song_per_artist=50)

song_df = pd.DataFrame(song_df)
song_df.to_csv("data/songs_2.csv", index=False)
song_df

Fetching song for artist:  Son Tung M-TP
Fetched 50 songs so far...
Fetching song for artist:  Den
Fetched 50 songs so far...
Fetching song for artist:  Vu.
Fetched 50 songs so far...
Fetching song for artist:  HIEUTHUHAI
Fetched 50 songs so far...
Fetching song for artist:  My Tam
Fetched 50 songs so far...
Fetching song for artist:  RAP VIET
Fetched 50 songs so far...
Fetching song for artist:  Low G
Fetched 50 songs so far...
Fetching song for artist:  W/N
Fetched 50 songs so far...
Fetching song for artist:  AMEE
Fetched 50 songs so far...
Fetching song for artist:  tlinh
Fetched 50 songs so far...
Fetching song for artist:  Duc Phuc
Fetched 50 songs so far...
Fetching song for artist:  Ha Anh Tuan
Fetched 50 songs so far...
Fetching song for artist:  Bich Phuong
Fetched 50 songs so far...
Fetching song for artist:  Binz
Fetched 50 songs so far...
Fetching song for artist:  RPT MCK
Fetched 50 songs so far...
Fetching song for artist:  Da LAB
Fetched 50 songs so far...
Fetching song

,id,name,artists,album,album_type,album_popularity,genres,available_markets,track_number,disc_number,duration_ms,popularity,release_date
0,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,[Sơn Tùng M-TP],Đừng Làm Trái Tim Anh Đau,single,55,"[v-pop, vinahouse, vietnamese hip hop]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,279208,64,2024-06-08
1,2wZAvkgiOE5tyrnqhB69KA,Âm Thầm Bên Em,[Sơn Tùng M-TP],m-tp M-TP,album,62,"[v-pop, vinahouse, vietnamese hip hop]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",11,1,293112,60,2017-04-01
2,3OVMe3H6iAxbLF8iD2UYrw,Chúng Ta Của Tương Lai,[Sơn Tùng M-TP],Chúng Ta Của Tương Lai,single,50,"[v-pop, vinahouse, vietnamese hip hop]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,249871,60,2024-03-08
3,4vfhartd7HDWGJbdVIycEu,Muộn Rồi Mà Sao Còn,[Sơn Tùng M-TP],Muộn Rồi Mà Sao Còn,single,48,"[v-pop, vinahouse, vietnamese hip hop]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,275905,58,2021-04-29
4,1J3SmWwlYAG68LGKr86MVH,Chúng Ta Của Hiện Tại,[Sơn Tùng M-TP],Chúng Ta Của Hiện Tại,single,49,"[v-pop, vinahouse, vietnamese hip hop]","[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,301538,59,2020-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,1SclLzmq4d95cLJu0plLP5,God Grace,[Liu.G],God Grace,single,0,[],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,201325,0,2023-01-22
4999,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),[Steve S. Liu],Amazing Grace (Music Box Version),single,0,[],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",1,1,183760,0,2023-10-31
5000,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,[Jingyi Liu],Moonlight Falls,album,0,[],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",11,1,94560,0,2024-12-20
5001,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,[Samuel Liu],Undiscovered Land,album,0,[],"[AR, AU, AT, BE, BO, BR, BG, CA, CL, CO, CR, C...",8,1,1315680,0,2019-01-24


In [ ]:
song_df = pd.read_csv("data/songs_2.csv")
song_df

,id,name,artists,album,album_type,album_popularity,genres,available_markets,track_number,disc_number,duration_ms,popularity,release_date
0,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,['Sơn Tùng M-TP'],Đừng Làm Trái Tim Anh Đau,single,55,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,279208,64,2024-06-08
1,2wZAvkgiOE5tyrnqhB69KA,Âm Thầm Bên Em,['Sơn Tùng M-TP'],m-tp M-TP,album,62,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,293112,60,2017-04-01
2,3OVMe3H6iAxbLF8iD2UYrw,Chúng Ta Của Tương Lai,['Sơn Tùng M-TP'],Chúng Ta Của Tương Lai,single,50,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,249871,60,2024-03-08
3,4vfhartd7HDWGJbdVIycEu,Muộn Rồi Mà Sao Còn,['Sơn Tùng M-TP'],Muộn Rồi Mà Sao Còn,single,48,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,275905,58,2021-04-29
4,1J3SmWwlYAG68LGKr86MVH,Chúng Ta Của Hiện Tại,['Sơn Tùng M-TP'],Chúng Ta Của Hiện Tại,single,49,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,301538,59,2020-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,1SclLzmq4d95cLJu0plLP5,God Grace,['Liu.G'],God Grace,single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,201325,0,2023-01-22
4999,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),['Steve S. Liu'],Amazing Grace (Music Box Version),single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,183760,0,2023-10-31
5000,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,['Jingyi Liu'],Moonlight Falls,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,94560,0,2024-12-20
5001,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,['Samuel Liu'],Undiscovered Land,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",8,1,1315680,0,2019-01-24


<a class="anchor" id="1.5."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.5. Get the missing features </b></h4>

In [8]:
songs_2 = pd.read_csv('data/songs_2.csv')

In [9]:
songs_2

,id,name,artists,album,album_type,album_popularity,genres,available_markets,track_number,disc_number,duration_ms,popularity,release_date
0,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,['Sơn Tùng M-TP'],Đừng Làm Trái Tim Anh Đau,single,55,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,279208,64,2024-06-08
1,2wZAvkgiOE5tyrnqhB69KA,Âm Thầm Bên Em,['Sơn Tùng M-TP'],m-tp M-TP,album,62,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,293112,60,2017-04-01
2,3OVMe3H6iAxbLF8iD2UYrw,Chúng Ta Của Tương Lai,['Sơn Tùng M-TP'],Chúng Ta Của Tương Lai,single,50,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,249871,60,2024-03-08
3,4vfhartd7HDWGJbdVIycEu,Muộn Rồi Mà Sao Còn,['Sơn Tùng M-TP'],Muộn Rồi Mà Sao Còn,single,48,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,275905,58,2021-04-29
4,1J3SmWwlYAG68LGKr86MVH,Chúng Ta Của Hiện Tại,['Sơn Tùng M-TP'],Chúng Ta Của Hiện Tại,single,49,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,301538,59,2020-12-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,1SclLzmq4d95cLJu0plLP5,God Grace,['Liu.G'],God Grace,single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,201325,0,2023-01-22
4999,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),['Steve S. Liu'],Amazing Grace (Music Box Version),single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,183760,0,2023-10-31
5000,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,['Jingyi Liu'],Moonlight Falls,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,94560,0,2024-12-20
5001,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,['Samuel Liu'],Undiscovered Land,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",8,1,1315680,0,2019-01-24


In [12]:
client_id = "b80f2ca2255d4d16b0fc73ce82aadb24"
client_secret = "322a7883a70f453cabf30401b04f66ca"
redirect_uri = "http://localhost:3000/callback"

In [18]:
auth_url = "https://accounts.spotify.com/authorize"

auth_params = {
    "client_id": client_id,
    "response_type": "code",
    "redirect_uri": redirect_uri,
    # "scope": scope
}

authorization_url = f"{auth_url}?{urllib.parse.urlencode(auth_params)}"
print("Go to the following URL to authorize the application:")
print(authorization_url)

Go to the following URL to authorize the application:
https://accounts.spotify.com/authorize?client_id=b80f2ca2255d4d16b0fc73ce82aadb24&response_type=code&redirect_uri=http%3A%2F%2Flocalhost%3A3000%2Fcallback


In [45]:
authorization_code = input("Enter the authorization code: ")

# Step 3: Exchange the authorization code for an access token and refresh token
token_url = "https://accounts.spotify.com/api/token"
credentials = f"{client_id}:{client_secret}".encode('utf-8')
encoded_credentials = base64.b64encode(credentials).decode('utf-8')

token_headers = {
    "Authorization": f"Basic {encoded_credentials}",
    "Content-Type": "application/x-www-form-urlencoded"
}

token_data = {
    "grant_type": "authorization_code",
    "code": authorization_code,
    "redirect_uri": redirect_uri
}

response = requests.post(token_url, headers=token_headers, data=token_data)

if response.status_code == 200:
    data = response.json()
    access_token = data["access_token"]
    refresh_token = data["refresh_token"]
    print("Access Token:", access_token)
    print("Refresh Token:", refresh_token)
else:
    print("Error:", response.status_code, response.text)

# Step 4: Use the refresh token to obtain a new access token
def refresh_access_token(client_id, client_secret, refresh_token):
    auth_url = 'https://accounts.spotify.com/api/token'
    
    credentials = f"{client_id}:{client_secret}".encode('utf-8')
    encoded_credentials = base64.b64encode(credentials).decode('utf-8')
    
    auth_header = {
        'Authorization': f'Basic {encoded_credentials}',
        'Content-Type': 'application/x-www-form-urlencoded'
    }
    
    auth_data = {
        'grant_type': 'refresh_token',
        'refresh_token': refresh_token
    }
    
    response = requests.post(auth_url, headers=auth_header, data=auth_data)
    
    if response.status_code == 200:
        data = response.json()
        return data['access_token']
    else:
        print("Error:", response.status_code, response.text)
        return None

# Use the refresh token to get a new access token
new_access_token = refresh_access_token(client_id, client_secret, refresh_token)
print("New Access Token:", new_access_token)

Error: 400 {"error":"invalid_client","error_description":"Invalid client"}
New Access Token: BQAAoKt3k1vQMYmWCyZgYF0jdjRqfbP2FRUc4Kc6BRNRECdviFiAGjvnkwTmpcuKXFDGuhJ-_bi7eomFmdTYAaYSLkdFkHE9CdC5UCWQD8G2QKtphwsTFlLF1ZXn4VYOSeDhjjHbPacQihwNQpuMni0SMjNRWeTTLyXGTKfWYzEz-XXbDKq_ZO8mu_G3OrEZz8JKZVHrtMBvfv1XdTwr6D-N9mSfXCAPPk0h4Q


In [46]:
new_access_token
sp = spotipy.Spotify(auth=new_access_token)

In [17]:

# Get the is_explicit field for each song
def get_explicit_field(track_id):
    track = sp.track(track_id)
    return track.get('explicit', None)

In [18]:
for i, row in songs_2.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 1 with ID 31VNCmwspR7nVJ6kruUuJt...
Explicit: False

Processing song 2 with ID 2wZAvkgiOE5tyrnqhB69KA...
Explicit: False

Processing song 3 with ID 3OVMe3H6iAxbLF8iD2UYrw...
Explicit: False

Processing song 4 with ID 4vfhartd7HDWGJbdVIycEu...
Explicit: False

Processing song 5 with ID 1J3SmWwlYAG68LGKr86MVH...
Explicit: False

Processing song 6 with ID 7ESH2ntyD5Bb1XckoJU554...
Explicit: False

Processing song 7 with ID 5rSAUmbOg4giDrM29vUyHe...
Explicit: False

Processing song 8 with ID 3P3rcI48wwVEgxF5vPM6pG...
Explicit: False

Processing song 9 with ID 6aaNsCMWPBaUuJE1grH6Po...
Explicit: False

Processing song 10 with ID 7bn8ckDTI4EQIKZ6jRL4K0...
Explicit: False

Processing song 11 with ID 4SqE2XLtd1AfppGm2xlkf4...
Explicit: False

Processing song 12 with ID 527UV7H9SbnmTVkxBVg56X...
Explicit: False

Processing song 13 with ID 0tyrLp1zVAjN3EV95EuOzQ...
Explicit: False

Processing song 14 with ID 0tPBIwb0p5mozyF6GZVTUG...
Explicit: False

Processing song 15 with ID 4Q

KeyboardInterrupt: 

In [19]:
songs_2

,id,name,artists,album,album_type,album_popularity,genres,available_markets,track_number,disc_number,duration_ms,popularity,release_date,explicit
0,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,['Sơn Tùng M-TP'],Đừng Làm Trái Tim Anh Đau,single,55,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,279208,64,2024-06-08,False
1,2wZAvkgiOE5tyrnqhB69KA,Âm Thầm Bên Em,['Sơn Tùng M-TP'],m-tp M-TP,album,62,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,293112,60,2017-04-01,False
2,3OVMe3H6iAxbLF8iD2UYrw,Chúng Ta Của Tương Lai,['Sơn Tùng M-TP'],Chúng Ta Của Tương Lai,single,50,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,249871,60,2024-03-08,False
3,4vfhartd7HDWGJbdVIycEu,Muộn Rồi Mà Sao Còn,['Sơn Tùng M-TP'],Muộn Rồi Mà Sao Còn,single,48,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,275905,58,2021-04-29,False
4,1J3SmWwlYAG68LGKr86MVH,Chúng Ta Của Hiện Tại,['Sơn Tùng M-TP'],Chúng Ta Của Hiện Tại,single,49,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,301538,59,2020-12-20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,1SclLzmq4d95cLJu0plLP5,God Grace,['Liu.G'],God Grace,single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,201325,0,2023-01-22,NaN
4999,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),['Steve S. Liu'],Amazing Grace (Music Box Version),single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,183760,0,2023-10-31,NaN
5000,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,['Jingyi Liu'],Moonlight Falls,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,94560,0,2024-12-20,NaN
5001,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,['Samuel Liu'],Undiscovered Land,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",8,1,1315680,0,2019-01-24,NaN


In [20]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [21]:
# Get the explicit of the next 1000 songs
songs_3 = songs_2[1002:2000].copy()

for i, row in songs_3.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)
    

Processing song 1003 with ID 6lNfj1VcaETCDfCuZJYj8I...
Explicit: False

Processing song 1004 with ID 49ykh7nLxxHkClY4wHFOt7...
Explicit: False

Processing song 1005 with ID 6F1ALV5RXd2Ghs7CNwYC6G...
Explicit: False

Processing song 1006 with ID 6xNed18guKRAOZMkvlxFA5...
Explicit: False

Processing song 1007 with ID 4z5dAyLMnkKSHkOkwbR1RX...
Explicit: False

Processing song 1008 with ID 4t8klWlToxvwHeXXA8SYpY...
Explicit: False

Processing song 1009 with ID 6d2SdcllPos7HflN5rXk76...
Explicit: False

Processing song 1010 with ID 1PgMKkHb6w1RdGhocmXS4d...
Explicit: False

Processing song 1011 with ID 7LqVJ4B2WlnfoMfNKRIJuq...
Explicit: False

Processing song 1012 with ID 6omqlTpk0xulezHygipOZx...
Explicit: False

Processing song 1013 with ID 3HCdFONw2wvrRsvJ5mcEDC...
Explicit: False

Processing song 1014 with ID 77agb6K1CMrwvTpC4k03Fm...
Explicit: False

Processing song 1015 with ID 31rqWdQbjz8x3JobYD1D5C...
Explicit: False

Processing song 1016 with ID 2NARD133RyXhJFMJIlgm7Y...
Explicit:

In [23]:
songs_2

,id,name,artists,album,album_type,album_popularity,genres,available_markets,track_number,disc_number,duration_ms,popularity,release_date,explicit
0,31VNCmwspR7nVJ6kruUuJt,Đừng Làm Trái Tim Anh Đau,['Sơn Tùng M-TP'],Đừng Làm Trái Tim Anh Đau,single,55,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,279208,64,2024-06-08,False
1,2wZAvkgiOE5tyrnqhB69KA,Âm Thầm Bên Em,['Sơn Tùng M-TP'],m-tp M-TP,album,62,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,293112,60,2017-04-01,False
2,3OVMe3H6iAxbLF8iD2UYrw,Chúng Ta Của Tương Lai,['Sơn Tùng M-TP'],Chúng Ta Của Tương Lai,single,50,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,249871,60,2024-03-08,False
3,4vfhartd7HDWGJbdVIycEu,Muộn Rồi Mà Sao Còn,['Sơn Tùng M-TP'],Muộn Rồi Mà Sao Còn,single,48,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,275905,58,2021-04-29,False
4,1J3SmWwlYAG68LGKr86MVH,Chúng Ta Của Hiện Tại,['Sơn Tùng M-TP'],Chúng Ta Của Hiện Tại,single,49,"['v-pop', 'vinahouse', 'vietnamese hip hop']","['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,301538,59,2020-12-20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4998,1SclLzmq4d95cLJu0plLP5,God Grace,['Liu.G'],God Grace,single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,201325,0,2023-01-22,NaN
4999,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),['Steve S. Liu'],Amazing Grace (Music Box Version),single,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",1,1,183760,0,2023-10-31,NaN
5000,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,['Jingyi Liu'],Moonlight Falls,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",11,1,94560,0,2024-12-20,NaN
5001,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,['Samuel Liu'],Undiscovered Land,album,0,[],"['AR', 'AU', 'AT', 'BE', 'BO', 'BR', 'BG', 'CA...",8,1,1315680,0,2019-01-24,NaN


In [22]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [24]:
songs_4 = songs_2[2000:3000].copy()
for i, row in songs_4.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 2001 with ID 5e9HXuKzLAFnlXGYeHVy8g...
Explicit: False

Processing song 2002 with ID 5eILSTERXSC5uJ86eezbpt...
Explicit: False

Processing song 2003 with ID 6tYKqXYgUn18wkPEbqyecF...
Explicit: False

Processing song 2004 with ID 5N9FD4PvFfzw0F1qu4BZWL...
Explicit: False

Processing song 2005 with ID 5bV1PnXi6B8kYl1OIUZKlu...
Explicit: False

Processing song 2006 with ID 1HDxv1ZL0AlbT13GTOOPn0...
Explicit: False

Processing song 2007 with ID 2iTzIxM3EcXXnWYyqnwbZ6...
Explicit: False

Processing song 2008 with ID 7G53KNMkppNKzbVDA4KhaK...
Explicit: False

Processing song 2009 with ID 4rKjuHonDIZjUM2h6honev...
Explicit: False

Processing song 2010 with ID 2TqjLbM2E5I4ce6iWTdXly...
Explicit: False

Processing song 2011 with ID 7mGnyV8iszJprTTUxD3aua...
Explicit: False

Processing song 2012 with ID 2KCWQyjcHQMGfTTnClEuGG...
Explicit: False

Processing song 2013 with ID 1QoEs95UcKglqpcYx6qam1...
Explicit: False

Processing song 2014 with ID 7iYjIqTrM9fdncKjuDVnOa...
Explicit:

HTTP Error for GET to https://api.spotify.com/v1/tracks/5KYv3kORBKUmVEoqbRRWxt with Params: {'market': None} returned 401 due to The access token expired


Processing song 2519 with ID 5KYv3kORBKUmVEoqbRRWxt...


SpotifyException: http status: 401, code:-1 - https://api.spotify.com/v1/tracks/5KYv3kORBKUmVEoqbRRWxt:
 The access token expired, reason: None

In [25]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [26]:
new_access_token = refresh_access_token(client_id, client_secret, refresh_token)
sp = spotipy.Spotify(auth=new_access_token)

In [27]:
songs_4_1 = songs_2[2520:3000].copy()
for i, row in songs_4_1.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 2521 with ID 6lx6unJJCy2a6Y8EqgTdvp...
Explicit: False

Processing song 2522 with ID 5mnSkB71UFNvOF1jPlHrWv...
Explicit: False

Processing song 2523 with ID 1DGeOHzwCxwJBiitcnUz0B...
Explicit: False

Processing song 2524 with ID 0SGu1JZK8j83QMERZxbUUy...
Explicit: False

Processing song 2525 with ID 3BGYJANZPG2KRneZKCzkxs...
Explicit: False

Processing song 2526 with ID 5oCS2wiDRVu0CcnNjzzwi9...
Explicit: False

Processing song 2527 with ID 0qZkd3StBLpuX0EEq3aVwU...
Explicit: False

Processing song 2528 with ID 4s9KzC1CduhQWbekLNCLLB...
Explicit: False

Processing song 2529 with ID 4nDM3iXzRzh3GkfXzC2zrX...
Explicit: False

Processing song 2530 with ID 2Tm2vf5JWQvhTPYkuTXbgW...
Explicit: False

Processing song 2531 with ID 70o0Td7UIuWsIy6tmehbXz...
Explicit: False

Processing song 2532 with ID 4QBkUrIr73JJygT03NTg2G...
Explicit: False

Processing song 2533 with ID 6tkHaG3Ih6GI8rw1g9ziOx...
Explicit: False

Processing song 2534 with ID 2xKnD3ZmHPmrpSt0gpR2EA...
Explicit:

KeyboardInterrupt: 

In [28]:
songs_4_1 = songs_2[2701:3000].copy()
for i, row in songs_4_1.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 2702 with ID 59cmKZYE8EYaKZOQTCIFtv...
Explicit: False

Processing song 2703 with ID 3SiyBOq1YCOjrQFY7evk6E...
Explicit: False

Processing song 2704 with ID 5AncfVq5nahFbbkzb2HAz7...
Explicit: False

Processing song 2705 with ID 3lpVmrNerAdO06BTxOeBhc...
Explicit: False

Processing song 2706 with ID 5DTbICIQ6SdU7zBMsPtCGr...
Explicit: False

Processing song 2707 with ID 6FcVB6xmkGUwtP0hpySRR2...
Explicit: False

Processing song 2708 with ID 4H232zNObSvj8vWoLxis0G...
Explicit: False

Processing song 2709 with ID 16rwZ0HnhJRKH97rJ3b2PK...
Explicit: False

Processing song 2710 with ID 3qRoVtlqvWwPzajo95XD5w...
Explicit: False

Processing song 2711 with ID 42mD1AOy93lxxJtVJhWD8e...
Explicit: False

Processing song 2712 with ID 0xAzmoItneUwyIJGoStrim...
Explicit: False

Processing song 2713 with ID 6v2GxIOlbtccYocypWDJoF...
Explicit: False

Processing song 2714 with ID 5NBlUxP0gz4DssRXe65cb8...
Explicit: False

Processing song 2715 with ID 2Gcls7eIysN3InhFrJYMo7...
Explicit:

In [29]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [30]:
songs_5 = songs_2[3000:4000].copy()
for i, row in songs_5.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 3001 with ID 6Dy4COn0KE2u4sZdWPckZd...
Explicit: True

Processing song 3002 with ID 1DEuRFIQSuekPkKVZvONrp...
Explicit: True

Processing song 3003 with ID 4gCZKHHxD70Cup8NhbKXSb...
Explicit: False

Processing song 3004 with ID 1avmx0YD7TeGh3P6j1fBf3...
Explicit: True

Processing song 3005 with ID 2gM6Ld7bA28ULuNh1fxCJv...
Explicit: True

Processing song 3006 with ID 0USWLczqP7TODzJEOZgUX0...
Explicit: True

Processing song 3007 with ID 6UNJT2rA1ktUOWrYA3LL41...
Explicit: True

Processing song 3008 with ID 6PcTELDNHxOIwR4HMYbrdn...
Explicit: False

Processing song 3009 with ID 1kXS3pUtSHx03gJ3N484kq...
Explicit: False

Processing song 3010 with ID 383R5MyXMSprWQRcZDUE5I...
Explicit: False

Processing song 3011 with ID 3Jpa2Req2UJarcOjTepyah...
Explicit: True

Processing song 3012 with ID 0hcULKe845tPs47Xoai1py...
Explicit: False

Processing song 3013 with ID 7CqVCyEencOoAbA45EnTqb...
Explicit: True

Processing song 3014 with ID 1xY93QVM6eiH4yLpiMpdBM...
Explicit: False



In [31]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [32]:
songs_6 = songs_2[4000:5002].copy()
for i, row in songs_6.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 4001 with ID 6UiaZlhrHDvOMT6takGh54...
Explicit: False

Processing song 4002 with ID 55qxvPDrw85Cnm0JimOLnC...
Explicit: False

Processing song 4003 with ID 0wfOrNziZpe1nEegGa8GsB...
Explicit: False

Processing song 4004 with ID 4I1jz4Ygm6ZBO6RpotZQFj...
Explicit: False

Processing song 4005 with ID 5uInJxP5jyl38XGlCaMrqw...
Explicit: False

Processing song 4006 with ID 4MLvQiVRNkSKR5VBeKAtYq...
Explicit: False

Processing song 4007 with ID 0SVdthFqRVlUiVitiWamWw...
Explicit: False

Processing song 4008 with ID 1utIo8IfdNczlv04OZKE64...
Explicit: False

Processing song 4009 with ID 3ShCMeq2gSOHgfVnw3tUXV...
Explicit: False

Processing song 4010 with ID 06Vmw0TFP8CnbXP7rOJOqW...
Explicit: False

Processing song 4011 with ID 5Ww2q3nwug2I2M2SNYmNSy...
Explicit: False

Processing song 4012 with ID 71EdTjInvFLzL3V8eQYwj3...
Explicit: False

Processing song 4013 with ID 3tJC3m7HJ6Y5XoKGEkc5vt...
Explicit: False

Processing song 4014 with ID 2GpTGeF2qvr826EFRzHUS3...
Explicit:

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

In [33]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [34]:
songs_6 = songs_2[4188:5002].copy()
for i, row in songs_6.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 4189 with ID 3kSbu5DFCwugVszEmtWdHR...
Explicit: False

Processing song 4190 with ID 5oaIMITSxQkzWe03fh2FUU...
Explicit: False

Processing song 4191 with ID 6yB5cd9Yl4Zq3Lm7lFh9aA...
Explicit: False

Processing song 4192 with ID 5075YemaytIvt4vFPKpRQQ...
Explicit: False

Processing song 4193 with ID 32ibn7IcX7KHfYmwvnEryi...
Explicit: False

Processing song 4194 with ID 1ojUtV3R0T0fKfFaAiCT2S...
Explicit: False

Processing song 4195 with ID 4m34TSYzhIelXoGQKul4bO...
Explicit: False

Processing song 4196 with ID 2oUtSlpf4u4ktK02nr76VR...
Explicit: False

Processing song 4197 with ID 2qtCOjcdcag0tVQXNMEebN...
Explicit: False

Processing song 4198 with ID 5Vf5Mc75rhCXnqL5hLVnCG...
Explicit: False

Processing song 4199 with ID 1XGoLfOaBGqrcl2VMSqnlt...
Explicit: False

Processing song 4200 with ID 26LA6GDDFvspPxNnXARFXH...
Explicit: False

Processing song 4201 with ID 57DEBZwRHZPCZegjhfKoYx...
Explicit: False

Processing song 4202 with ID 204d4j2COaNASmJatiNgPL...
Explicit:

In [35]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [37]:
# Get the last row
songs_6 = songs_2[5002:].copy()

for i, row in songs_6.iterrows():
    track_id = row['id']
    print(f"Processing song {i + 1} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 5003 with ID 1HFxjJAiU0baHU8kTbwe5R...
Explicit: False



In [38]:
songs_2.to_csv('data/songs_2_add_explicit.csv', index=False)

In [43]:
# Get the rows that have not been processed
missing = songs_2[songs_2['explicit'].isnull()]['id']

In [ ]:
for i, track_id in missing.items():
    print(f"Processing song {i} with ID {track_id}...")
    explicit = get_explicit_field(track_id)
    print(f"Explicit: {explicit}")
    print()
    songs_2.loc[i, 'explicit'] = explicit
    time.sleep(1)

Processing song 1000 with ID 6H1SBYLM9WUHhK33xUEO90...
Explicit: False

Processing song 1001 with ID 2WvUmmJunRSb5Q1KpTg2N3...
Explicit: False

Processing song 2518 with ID 5KYv3kORBKUmVEoqbRRWxt...
Explicit: False



Processing song 2519 with ID 0IbSn77m6lTkFyu6zKn7Fd...


<a class="anchor" id="1.6."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.6. Get added genres </b></h4>


In [ ]:
import os
import requests
from flask import Flask, session
from spotipy import Spotify
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.cache_handler import CacheHandler
import base64
from spotipy.exceptions import SpotifyException
from unidecode import unidecode

import pandas as pd
import numpy as np
import time
import spotipy

import time
from spotipy.oauth2 import SpotifyClientCredentials
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

import ast

In [ ]:
client_id = "c904a86c09244b8386fd0501a7f3b74b"
client_secret = "268a3c41b1a74ac1b5c41923a9931fe2"

client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

access_token = client_credentials_manager.get_access_token(as_dict=False)

print("Access Token:", access_token)

Access Token: BQCyB4-l1I6tpDHoz6QC_v0QKTBa37Qzf4ffvpa72_rkZZnwe2-vMtx0KZsl340GDXFuH7Ltfu3BQKHeShw0AzcvFlD-cW8ogqhcmt2MTAfIiHj-zqcNi_UWwWa_XQ8MGdujWp_Is0c


In [ ]:
song_df = pd.read_csv("vietnamese_songs_2.csv")

In [ ]:
CHECKPOINT_FILE = "genres_checkpoint.csv"
artist_genre_cache = {}
genre_data = {}


def get_artist_genres_from_track(track_id):
    print("Getting genre for track: ", track_id)
    if track_id in genre_data:
        return genre_data[track_id]

    try:
        track_info = sp.track(track_id)
        artist_ids = [artist['id'] for artist in track_info['artists']]

        genres = set()
        for artist_id in artist_ids:
            if artist_id in artist_genre_cache:
                genres.update(artist_genre_cache[artist_id])
            else:
                artist_info = sp.artist(artist_id)
                artist_genre_cache[artist_id] = artist_info.get('genres', [])
                genres.update(artist_genre_cache[artist_id])

        genre_list = list(genres)
        genre_data[track_id] = genre_list

        df = pd.DataFrame({'id': [track_id], 'genres': [str(genre_list)]})
        df.to_csv(CHECKPOINT_FILE, mode='a', header=not os.path.exists(CHECKPOINT_FILE), index=False)

        return genre_list

    except spotipy.exceptions.SpotifyException as e:
        print(f"Error fetching track {track_id}: {e}")
        return []

In [ ]:
song_df['genres'] = song_df['id'].apply(lambda x: get_artist_genres_from_track(x))

Getting genre for track:  7LVrrX8pkzI9fMF88nGabt
Getting genre for track:  0ReId5SRMspb6ISw0rBsBN
Getting genre for track:  0s7RyyUlQfd8mnnboHe18n
Getting genre for track:  0eFMbKCRw8KByXyWBw8WO7


KeyboardInterrupt: 

In [ ]:
song_df.drop(columns='genres', inplace=True)
song_df

,album,album_type,album_popularity,artists,available_markets,disc_number,duration_ms,explicit,id,is_local,is_playable,name,popularity,track_number,type,release_date
0,Đi Về Nhà,single,50,"['Đen', 'JustaTee']",185,1,200000,False,7LVrrX8pkzI9fMF88nGabt,False,True,Đi Về Nhà,61,1,track,2020-12-20
1,CUA,single,40,"['MANBO', 'HIEUTHUHAI']",185,1,193475,True,0ReId5SRMspb6ISw0rBsBN,False,True,CUA,50,1,track,2020-04-05
2,Hẹn Gặp Em Dưới Ánh Trăng,single,53,"['MANBO', 'HIEUTHUHAI', 'HURRYKNG']",185,1,222000,False,0s7RyyUlQfd8mnnboHe18n,False,True,Hẹn Gặp Em Dưới Ánh Trăng,63,1,track,2020-09-25
3,Still With You,single,64,['Jung Kook'],185,1,239905,False,0eFMbKCRw8KByXyWBw8WO7,False,True,Still With You,75,1,track,2020-06-05
4,HELP EVER HURT NEVER,album,65,['Fujii Kaze'],179,1,185573,False,0o9zmvc5f3EFApU52PPIyW,False,True,Shinunoga E-Wa,74,8,track,2020-05-20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,külm,single,31,['An-Marlen'],185,1,176200,False,1M00EDfnI8zrf0clHUSJY6,False,True,külm,42,1,track,2024-12-06
4856,SUEÑO MAGNIFICO,single,48,['Najm'],185,1,136968,True,6x4eiL3u5iWWDVumT8JSqd,False,True,LOTSOFLOVE,51,4,track,2024-09-05
4857,Alleen Met Jou,single,56,"['Yves Berendse', 'Emma Heesters']",183,1,193694,False,2K7HqkHEUfz7c4BzEO33Qd,False,True,Alleen Met Jou,66,1,track,2024-09-27
4858,TUTTA VITA,album,72,"['Olly', 'JVLI']",185,1,190122,False,1ej2NcW0CHnscPp1RZpr1V,False,True,Devastante,68,6,track,2024-10-25


In [ ]:
genres_df = pd.read_csv('genres_checkpoint.csv')
genres_df

,id,genres
0,7LVrrX8pkzI9fMF88nGabt,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
1,0ReId5SRMspb6ISw0rBsBN,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
2,0s7RyyUlQfd8mnnboHe18n,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
3,0eFMbKCRw8KByXyWBw8WO7,['k-pop']
4,0o9zmvc5f3EFApU52PPIyW,['j-pop']
...,...,...
4855,1M00EDfnI8zrf0clHUSJY6,[]
4856,6x4eiL3u5iWWDVumT8JSqd,[]
4857,2K7HqkHEUfz7c4BzEO33Qd,['nederpop']
4858,1ej2NcW0CHnscPp1RZpr1V,[]


In [ ]:
merged_df = song_df.merge(genres_df, on='id', how='left')
merged_df

,album,album_type,album_popularity,artists,available_markets,disc_number,duration_ms,explicit,id,is_local,is_playable,name,popularity,track_number,type,release_date,genres
0,Đi Về Nhà,single,50,"['Đen', 'JustaTee']",185,1,200000,False,7LVrrX8pkzI9fMF88nGabt,False,True,Đi Về Nhà,61,1,track,2020-12-20,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
1,CUA,single,40,"['MANBO', 'HIEUTHUHAI']",185,1,193475,True,0ReId5SRMspb6ISw0rBsBN,False,True,CUA,50,1,track,2020-04-05,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
2,Hẹn Gặp Em Dưới Ánh Trăng,single,53,"['MANBO', 'HIEUTHUHAI', 'HURRYKNG']",185,1,222000,False,0s7RyyUlQfd8mnnboHe18n,False,True,Hẹn Gặp Em Dưới Ánh Trăng,63,1,track,2020-09-25,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
3,Still With You,single,64,['Jung Kook'],185,1,239905,False,0eFMbKCRw8KByXyWBw8WO7,False,True,Still With You,75,1,track,2020-06-05,['k-pop']
4,HELP EVER HURT NEVER,album,65,['Fujii Kaze'],179,1,185573,False,0o9zmvc5f3EFApU52PPIyW,False,True,Shinunoga E-Wa,74,8,track,2020-05-20,['j-pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,külm,single,31,['An-Marlen'],185,1,176200,False,1M00EDfnI8zrf0clHUSJY6,False,True,külm,42,1,track,2024-12-06,[]
4856,SUEÑO MAGNIFICO,single,48,['Najm'],185,1,136968,True,6x4eiL3u5iWWDVumT8JSqd,False,True,LOTSOFLOVE,51,4,track,2024-09-05,[]
4857,Alleen Met Jou,single,56,"['Yves Berendse', 'Emma Heesters']",183,1,193694,False,2K7HqkHEUfz7c4BzEO33Qd,False,True,Alleen Met Jou,66,1,track,2024-09-27,['nederpop']
4858,TUTTA VITA,album,72,"['Olly', 'JVLI']",185,1,190122,False,1ej2NcW0CHnscPp1RZpr1V,False,True,Devastante,68,6,track,2024-10-25,[]


In [ ]:
merged_df.to_csv("vietnamese_song_added_genres.csv", index=False)


In [ ]:
song_df = pd.read_csv("vietnamese_song_added_genres.csv")



In [ ]:
song_df

,album,album_type,album_popularity,artists,available_markets,disc_number,duration_ms,explicit,id,is_local,is_playable,name,popularity,track_number,type,release_date,genres
0,Đi Về Nhà,single,50,"['Đen', 'JustaTee']",185,1,200000,False,7LVrrX8pkzI9fMF88nGabt,False,True,Đi Về Nhà,61,1,track,2020-12-20,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
1,CUA,single,40,"['MANBO', 'HIEUTHUHAI']",185,1,193475,True,0ReId5SRMspb6ISw0rBsBN,False,True,CUA,50,1,track,2020-04-05,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
2,Hẹn Gặp Em Dưới Ánh Trăng,single,53,"['MANBO', 'HIEUTHUHAI', 'HURRYKNG']",185,1,222000,False,0s7RyyUlQfd8mnnboHe18n,False,True,Hẹn Gặp Em Dưới Ánh Trăng,63,1,track,2020-09-25,"['v-pop', 'vietnamese hip hop', 'vietnam indie..."
3,Still With You,single,64,['Jung Kook'],185,1,239905,False,0eFMbKCRw8KByXyWBw8WO7,False,True,Still With You,75,1,track,2020-06-05,['k-pop']
4,HELP EVER HURT NEVER,album,65,['Fujii Kaze'],179,1,185573,False,0o9zmvc5f3EFApU52PPIyW,False,True,Shinunoga E-Wa,74,8,track,2020-05-20,['j-pop']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,külm,single,31,['An-Marlen'],185,1,176200,False,1M00EDfnI8zrf0clHUSJY6,False,True,külm,42,1,track,2024-12-06,[]
4856,SUEÑO MAGNIFICO,single,48,['Najm'],185,1,136968,True,6x4eiL3u5iWWDVumT8JSqd,False,True,LOTSOFLOVE,51,4,track,2024-09-05,[]
4857,Alleen Met Jou,single,56,"['Yves Berendse', 'Emma Heesters']",183,1,193694,False,2K7HqkHEUfz7c4BzEO33Qd,False,True,Alleen Met Jou,66,1,track,2024-09-27,['nederpop']
4858,TUTTA VITA,album,72,"['Olly', 'JVLI']",185,1,190122,False,1ej2NcW0CHnscPp1RZpr1V,False,True,Devastante,68,6,track,2024-10-25,[]


In [ ]:
song_df = pd.read_csv("vietnamese_song_added_genres.csv")

In [ ]:
copy_df  = song_df.loc[4236:].copy()
copy_df


,album,album_type,album_popularity,artists,available_markets,disc_number,duration_ms,explicit,id,is_local,is_playable,name,popularity,track_number,type,release_date,genres
4236,Oh Garota Eu Quero Você Só Pra Mim,single,73,"['Oruam', 'Zé Felipe', 'MC Tuto', 'Dj Lc da Ro...",185,1,166861,True,0BUkQGUXgcvf3gdCWYbtnJ,False,True,Oh Garota Eu Quero Você Só Pra Mim,83,1,track,2024-12-17,NaN
4237,Anti ∞ Hero,single,58,['Mighfar Suganda'],185,1,210634,False,2GO1pRF96wqjB5TKXLQ0vY,False,True,Anti ∞ Hero,68,1,track,2024-10-24,NaN
4238,APOCALYPSE,album,71,['Gazo'],185,1,217693,True,0qlYaRFGvgR6afy1f4BBwD,False,True,NANANI NANANA,77,5,track,2024-11-29,NaN
4239,LE NORD SE SOUVIENT,album,69,['GIMS'],185,1,165978,False,0yyFAbTlOk7DFhh6RSug9S,False,True,SOIS PAS TIMIDE,73,3,track,2024-09-13,NaN
4240,Sadrah,single,59,['For Revenge'],185,1,247007,False,33UIshRVxXEBzITjCzo6al,False,True,Sadrah,69,1,track,2024-03-18,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4855,külm,single,31,['An-Marlen'],185,1,176200,False,1M00EDfnI8zrf0clHUSJY6,False,True,külm,42,1,track,2024-12-06,NaN
4856,SUEÑO MAGNIFICO,single,48,['Najm'],185,1,136968,True,6x4eiL3u5iWWDVumT8JSqd,False,True,LOTSOFLOVE,51,4,track,2024-09-05,NaN
4857,Alleen Met Jou,single,56,"['Yves Berendse', 'Emma Heesters']",183,1,193694,False,2K7HqkHEUfz7c4BzEO33Qd,False,True,Alleen Met Jou,66,1,track,2024-09-27,NaN
4858,TUTTA VITA,album,72,"['Olly', 'JVLI']",185,1,190122,False,1ej2NcW0CHnscPp1RZpr1V,False,True,Devastante,68,6,track,2024-10-25,NaN


In [ ]:
copy_df['genres'] = copy_df['id'].apply(lambda x: get_artist_genres_from_track(x))

Getting genre for track:  0BUkQGUXgcvf3gdCWYbtnJ
Getting genre for track:  2GO1pRF96wqjB5TKXLQ0vY
Getting genre for track:  0qlYaRFGvgR6afy1f4BBwD
Getting genre for track:  0yyFAbTlOk7DFhh6RSug9S
Getting genre for track:  33UIshRVxXEBzITjCzo6al
Getting genre for track:  5NWpquAje6NyvxD57oXNu6
Getting genre for track:  3GdN5n6p34yFMd12WKq8jL
Getting genre for track:  6a9Z1jUms915w4O7N1PxjY
Getting genre for track:  5eBk8ZXcd0pb0AO4a5PpOg
Getting genre for track:  4jUZymsR3lkia9wkP0o9cX
Getting genre for track:  7ricDBUakN3N0YkkKE8Obu
Getting genre for track:  7sditHCsoFw3nHnjFv1BHV
Getting genre for track:  3OJDtCdYs8QJnui708tEU3
Getting genre for track:  7Fzl7QaTu47WyP9R5S5mh5
Getting genre for track:  3ptRODxVj328j6TnW33Zj3
Getting genre for track:  5Nwh8kghoqebxmco5ZWbDt
Getting genre for track:  2gsRMFzelZuRQo8r11RsLr
Getting genre for track:  1qxKva4IzWde7m2jLpZvDU
Getting genre for track:  5H4MSA6eIr6x1iBTn1vpMG
Getting genre for track:  3zmN19fLAcKeDaajrIdWLB
Getting genre for tr

In [ ]:
genre_data

{'3T4DdPVqWeaC4U7vrIxlKz': ['schlager', 'schlagerparty'],
 '6TK7jseRCrogw9IzQYkw2h': ['tollywood',
  'tamil hip hop',
  'tamil dance',
  'tamil pop',
  'kannada pop',
  'telugu pop',
  'kollywood'],
 '4vIbws1xp082bv0zl8vLaJ': ['egyptian pop',
  'khaleeji',
  'mahraganat',
  'egyptian hip hop',
  'arabic rap']}

<a class="anchor" id="1.7."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.7. Get stream counts </b></h4>



In [ ]:
songs = pd.read_csv('data/vietnamese_songs.csv')
songs_2 = pd.read_csv('data/songs_2.csv')

In [ ]:
def get_stream_counts(songs):
    # Initialize Selenium WebDriver
    options = Options()
    options.headless = True  # Run in headless mode
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)

    view_counts= []
    found = 0

    for i in range(len(songs)):
        song = songs.iloc[i]
        song_name = song['name']
        artist = song['artists']
        id = song['id']
        
        print(f"Processing ID: {id}")
        url = f"https://www.mystreamcount.com/track/{id}"
        driver.get(url)
        
        stream_counts_text = None
        attempts = 0
        max_attempts = 3
        
        while attempts < max_attempts and stream_counts_text is None:
            try:
                # Wait for the stream counts element to be present
                stream_counts_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, 'span.text-lg.font-bold.underline'))
                )
                stream_counts_text = stream_counts_element.text.strip()
            except Exception as e:
                print(f"Stream counts not found for id {id} on attempt {attempts + 1}: {e}")
                attempts += 1
                if attempts < max_attempts:
                    driver.refresh()
                    time.sleep(2)  # Wait a bit before retrying
        
        if stream_counts_text is None:
            view_counts.append({
                'id'   : id,
                'name': song_name,
                'artists': artist,
                'stream_counts': None
            })
        else:
            print(f"Stream counts text: {stream_counts_text}")
            
            if "less than 1000" in stream_counts_text.lower():
                stream_counts_value = "< 1000"  # Set to a string "< 1000"
            else:
                try:
                    stream_counts_value = int(stream_counts_text.replace(',', ''))
                except ValueError:
                    print(f"Failed to convert stream counts to integer for id {id}: {stream_counts_text}")
                    stream_counts_value = None
            
            view_counts.append({
                'id'   : id,
                'name': song_name,
                'artists': artist,
                'stream_counts': stream_counts_value
            })
            found += 1
            print(f"No of found songs: {found}")
        
        time.sleep(1)

    driver.quit()
    return view_counts

In [ ]:
view_counts = get_stream_counts(songs)

Processing ID: 7LVrrX8pkzI9fMF88nGabt
Stream counts text: 40,920,596
No of found songs: 1
Processing ID: 0ReId5SRMspb6ISw0rBsBN
Stream counts text: 13,677,572
No of found songs: 2
Processing ID: 0s7RyyUlQfd8mnnboHe18n
Stream counts text: 26,714,031
No of found songs: 3
Processing ID: 0eFMbKCRw8KByXyWBw8WO7
Stream counts text: 272,565,438
No of found songs: 4
Processing ID: 0o9zmvc5f3EFApU52PPIyW
Stream counts text: 656,179,826
No of found songs: 5
Processing ID: 4HBZA5flZLE435QTztThqH
Stream counts text: 1,233,247,635
No of found songs: 6
Processing ID: 72IwoG8tqvIWV10IHjpNNA
Stream counts text: 2,024,180,031
No of found songs: 7
Processing ID: 4inMQ83GNpQ2OHDredH5hW
Stream counts text: 13,689,452
No of found songs: 8
Processing ID: 0VjIjW4GlUZAMYd2vXMi3b
Stream counts text: 4,293,744,430
No of found songs: 9
Processing ID: 3USxtqRwSYz57Ewm6wWRMp
Stream counts text: 3,296,021,376
No of found songs: 10
Processing ID: 6FZDfxM3a3UCqtzo5pxSLZ
Stream counts text: 2,197,889,244
No of found s

In [ ]:
view_counts_2 = get_stream_counts(songs_2)

Processing ID: 31VNCmwspR7nVJ6kruUuJt
Stream counts text: 40,020,316
No of found songs: 1
Processing ID: 2wZAvkgiOE5tyrnqhB69KA
Stream counts text: 33,029,412
No of found songs: 2
Processing ID: 3OVMe3H6iAxbLF8iD2UYrw
Stream counts text: 29,533,035
No of found songs: 3
Processing ID: 4vfhartd7HDWGJbdVIycEu
Stream counts text: 49,953,175
No of found songs: 4
Processing ID: 1J3SmWwlYAG68LGKr86MVH
Stream counts text: 58,417,877
No of found songs: 5
Processing ID: 7ESH2ntyD5Bb1XckoJU554
Stream counts text: 8,246,014
No of found songs: 6
Processing ID: 5rSAUmbOg4giDrM29vUyHe
Stream counts text: 31,967,209
No of found songs: 7
Processing ID: 3P3rcI48wwVEgxF5vPM6pG
Stream counts text: 20,547,598
No of found songs: 8
Processing ID: 6aaNsCMWPBaUuJE1grH6Po
Stream counts text: 31,243,253
No of found songs: 9
Processing ID: 7bn8ckDTI4EQIKZ6jRL4K0
Stream counts text: 13,011,000
No of found songs: 10
Processing ID: 4SqE2XLtd1AfppGm2xlkf4
Stream counts text: 6,749,789
No of found songs: 11
Processing

In [ ]:
view_counts = pd.DataFrame(view_counts)
view_counts_2 = pd.DataFrame(view_counts_2)

In [ ]:
view_counts.to_csv('data/vietnamese_songs_stream_counts.csv', index=False)
view_counts_2.to_csv('data/songs_2_stream_counts.csv', index=False)

<a class="anchor" id="1.8."></a>
<div class="list-group" id="list-tab" role="tablist">
    <h4 style="text-align: left; background-color:rgb(36, 138, 193); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>1.8. Merging datasets </b></h4>



In [40]:
df_1 = pd.read_csv("data/vietnamese_song_added_genres.csv")
df_1_streamcounts = pd.read_csv("data/vietnamese_songs_stream_counts.csv")
df_1_streamcounts = df_1_streamcounts[['id','stream_counts']]


df_2 = pd.read_csv("data/songs_2_add_explicit.csv")
df_2_streamcounts = pd.read_csv("data/songs_2_stream_counts.csv")
df_2_streamcounts = df_2_streamcounts[['id','stream_counts']]


In [41]:
df_1.drop(columns=["is_local", "is_playable", "type", "available_markets"], inplace=True) # Drop uneccesary columns
merged_df_1 = df_1.merge(df_1_streamcounts, on='id', how='left')


In [42]:
df_2.drop(columns="available_markets", inplace=True)
merged_df_2 = df_2.merge(df_2_streamcounts, on='id', how='left')

In [64]:
df_full = pd.concat([merged_df_1, merged_df_2], ignore_index=True)
df_full = df_full.drop_duplicates(subset='id', keep='first').reset_index(drop=True)

df_full

,album,album_type,album_popularity,artists,disc_number,duration_ms,explicit,id,name,popularity,track_number,release_date,genres,stream_counts
0,Đi Về Nhà,single,50,"['Đen', 'JustaTee']",1,200000,False,7LVrrX8pkzI9fMF88nGabt,Đi Về Nhà,61,1,2020-12-20,"['v-pop', 'vietnamese hip hop', 'vietnam indie...",40920596.0
1,CUA,single,40,"['MANBO', 'HIEUTHUHAI']",1,193475,True,0ReId5SRMspb6ISw0rBsBN,CUA,50,1,2020-04-05,"['v-pop', 'vietnamese hip hop', 'vietnam indie...",13677572.0
2,Hẹn Gặp Em Dưới Ánh Trăng,single,53,"['MANBO', 'HIEUTHUHAI', 'HURRYKNG']",1,222000,False,0s7RyyUlQfd8mnnboHe18n,Hẹn Gặp Em Dưới Ánh Trăng,63,1,2020-09-25,"['v-pop', 'vietnamese hip hop', 'vietnam indie...",26714031.0
3,Still With You,single,64,['Jung Kook'],1,239905,False,0eFMbKCRw8KByXyWBw8WO7,Still With You,75,1,2020-06-05,['k-pop'],272565438.0
4,HELP EVER HURT NEVER,album,65,['Fujii Kaze'],1,185573,False,0o9zmvc5f3EFApU52PPIyW,Shinunoga E-Wa,74,8,2020-05-20,['j-pop'],656179826.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9714,God Grace,single,0,['Liu.G'],1,201325,False,1SclLzmq4d95cLJu0plLP5,God Grace,0,1,2023-01-22,[],NaN
9715,Amazing Grace (Music Box Version),single,0,['Steve S. Liu'],1,183760,False,5I9dn9tLgahzYktOobZKSn,Amazing Grace (Music Box Version),0,1,2023-10-31,[],NaN
9716,Moonlight Falls,album,0,['Jingyi Liu'],1,94560,False,3PtUcmIMK7s0vUKOo6gGXO,Dancing Gracefully,0,11,2024-12-20,[],NaN
9717,Undiscovered Land,album,0,['Samuel Liu'],1,1315680,False,3Ma6FipGlFRSMMewJFqHcK,Fields of Grace - Entering In,0,8,2019-01-24,[],NaN


In [63]:
df_full.to_csv("data/vietnamese_songs_full.csv",index=False)

<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
    <h3 style="text-align: center; background-color:rgb(2, 100, 153); font-family:newtimeroman; color: black; padding: 14px; line-height: 1; border-radius:10px"><b>2. Exploring and Pre-processing Data </b></h3>